<a href="https://colab.research.google.com/github/rajesh98/my-first-blog/blob/master/AntiSoof--real-fake%20face%20detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
print(os.getcwd())

/content


In [2]:
import numpy as np
import cv2
from scipy import ndimage

In [72]:
net = cv2.dnn.readNetFromCaffe('/content/sample_data/deploy.prototxt.txt', '/content/sample_data/openCVfaceDetectorModel1.caffemodel')

In [73]:
vs = cv2.VideoCapture('/content/sample_data/MakePeopleSmileProject.mp4')

In [74]:
vs2 = cv2.VideoCapture('/content/sample_data/MakePeopleSmileProjectFAKE.mp4')

In [75]:
read = 0
saved = 0

In [76]:
while True:
  #print("start")
  (grabbed,frame) = vs2.read()
  if not grabbed:
    break
  read = read+1
  if read%9 !=0:
    continue
  (h,w) = frame.shape[:2]
  blob = cv2.dnn.blobFromImage(cv2.resize(frame,(300,300)),1.0,(300,300),(104.0,177.0,123.0))
  net.setInput(blob)
  detections = net.forward()
  #print(len(detections))
  if len(detections>0):
    i = np.argmax(detections[0,0,:,2])
    confidence = detections[0,0,i,2]
    print(confidence)
    if confidence>0.5:
      box = detections[0,0,i,3:7] * np.array([w,h,w,h])
      (startX,startY,endX,endY) = box.astype("int")
      face = frame[startY:endY, startX:endX]
      rotated = ndimage.rotate(face, -90)
      op = os.path.sep.join(['/content/sample_data/dataset/fake', "{}.png".format(saved)])
      try:
        cv2.imwrite(op,rotated)
      except:
        print("error")

      saved = saved+1


0.52433187
0.6790611
0.43499672
0.39483902
0.3060459
0.26415727
0.94581294
0.87593246
0.9603117
0.9565402
0.9675134
0.9055092
0.94372505
0.9617667
0.95012295
0.9555995
0.9709692
0.96416336
0.9832548
0.46959525
0.37060103
0.58204985
0.57881564
0.679513
0.6277932
0.2077564
0.22706257
0.2442976
0.22100902
0.2470604
0.27067387
0.17740081
0.99643207
0.9899267
0.95620275
0.9922174
0.97621626
0.99216014
0.97827303
0.33513165
0.4070788
0.39042202
0.32429826
0.14586538
0.30099565
0.36289728
0.5972153
0.6262619
0.56985086
0.62462825
0.7214305
0.58641404
0.77624
0.58214855
0.47721544
0.48350617
0.13278697
0.13191596
0.14345072
0.1326012
0.13697173
0.14592086
0.13716486
0.5147828
0.6039734
0.6422162
0.6056104
0.6456888
0.7044872
0.8579336
0.744902
0.77613974
0.77381855
0.77732563
0.5890916
0.13440141
0.13614292
0.13750128
0.13639861
0.13641576
0.1353547
0.13659684
0.13782543
0.1401389
0.13629048
0.13805276
0.13648194
0.13657515
0.14234301
0.19346127
0.24556749
0.3652567
0.27544442
0.5523934
0.6633

In [77]:
print(read)
print(saved)

1126
68


In [111]:
# import the necessary packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as K

In [112]:
class LivenessNet:
	@staticmethod
	def build(width, height, depth, classes):
		# initialize the model along with the input shape to be
		# "channels last" and the channels dimension itself
		model = Sequential()
		inputShape = (height, width, depth)
		chanDim = -1
		# if we are using "channels first", update the input shape
		# and channels dimension
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)
			chanDim = 1

        # first CONV => RELU => CONV => RELU => POOL layer set
		model.add(Conv2D(16, (3, 3), padding="same",
			input_shape=inputShape))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(16, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))

        # second CONV => RELU => CONV => RELU => POOL layer set
		model.add(Conv2D(32, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(32, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))

        # first (and only) set of FC => RELU layers
		model.add(Flatten())
		model.add(Dense(64))
		model.add(Activation("relu"))
		model.add(BatchNormalization())
		model.add(Dropout(0.5))
		# softmax classifier
		model.add(Dense(classes))
		model.add(Activation("softmax"))

        # return the constructed network architecture
		return model

In [113]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
# import the necessary packages
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import cv2
import os

In [114]:
# initialize the initial learning rate, batch size, and number of
# epochs to train for
INIT_LR = 1e-4
BS = 8
EPOCHS = 50
# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")
imagePaths = list(paths.list_images('/content/sample_data/dataset'))
data = []
labels = []
# loop over all image paths
for imagePath in imagePaths:
	# extract the class label from the filename, load the image and
	# resize it to be a fixed 32x32 pixels, ignoring aspect ratio
  print(imagePath)
  label = imagePath.split(os.path.sep)[-2]
  print(label)
  image = cv2.imread(imagePath)
  image = cv2.resize(image, (32, 32))
	# update the data and labels lists, respectively
  data.append(image)
  labels.append(label)
# convert the data into a NumPy array, then preprocess it by scaling
# all pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0

[INFO] loading images...
/content/sample_data/dataset/real/172.png
real
/content/sample_data/dataset/real/26.png
real
/content/sample_data/dataset/real/27.png
real
/content/sample_data/dataset/real/127.png
real
/content/sample_data/dataset/real/73.png
real
/content/sample_data/dataset/real/98.png
real
/content/sample_data/dataset/real/14.png
real
/content/sample_data/dataset/real/44.png
real
/content/sample_data/dataset/real/168.png
real
/content/sample_data/dataset/real/23.png
real
/content/sample_data/dataset/real/60.png
real
/content/sample_data/dataset/real/126.png
real
/content/sample_data/dataset/real/79.png
real
/content/sample_data/dataset/real/137.png
real
/content/sample_data/dataset/real/158.png
real
/content/sample_data/dataset/real/24.png
real
/content/sample_data/dataset/real/10.png
real
/content/sample_data/dataset/real/153.png
real
/content/sample_data/dataset/real/130.png
real
/content/sample_data/dataset/real/181.png
real
/content/sample_data/dataset/real/83.png
real


In [91]:
# encode the labels (which are currently strings) as integers and then
# one-hot encode them
le = LabelEncoder()
labels = le.fit_transform(labels)
labels = to_categorical(labels, 2)
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.25, random_state=42)

In [115]:
print(len(data))

240


In [116]:
# construct the training image generator for data augmentation
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15,
	width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,
	horizontal_flip=True, fill_mode="nearest")
# initialize the optimizer and model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model = LivenessNet.build(width=32, height=32, depth=3,
	classes=len(le.classes_))
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])
# train the network
print("[INFO] training network for {} epochs...".format(EPOCHS))
H = model.fit(x=aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,
	epochs=EPOCHS)

[INFO] compiling model...
[INFO] training network for 50 epochs...


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/50
22/22 [==============================] - 2s 46ms/step - loss: 0.9449 - accuracy: 0.5581 - val_loss: 0.6666 - val_accuracy: 0.6333
Epoch 2/50
22/22 [==============================] - 1s 34ms/step - loss: 0.6975 - accuracy: 0.7442 - val_loss: 0.6450 - val_accuracy: 0.6333
Epoch 3/50
22/22 [==============================] - 1s 35ms/step - loss: 0.5929 - accuracy: 0.7907 - val_loss: 0.6423 - val_accuracy: 0.6333
Epoch 4/50
22/22 [==============================] - 1s 33ms/step - loss: 0.5380 - accuracy: 0.7791 - val_loss: 0.6444 - val_accuracy: 0.6333
Epoch 5/50
22/22 [==============================] - 1s 34ms/step - loss: 0.4917 - accuracy: 0.8314 - val_loss: 0.6429 - val_accuracy: 0.6333
Epoch 6/50
22/22 [==============================] - 1s 36ms/step - loss: 0.4695 - accuracy: 0.8198 - val_loss: 0.6492 - val_accuracy: 0.6833
Epoch 7/50
22/22 [==============================] - 1s 36ms/step - loss: 0.4322 - accuracy: 0.8488 - val_loss: 0.6416 - val_accuracy: 0.7667
Epoch 8/50
22

In [117]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(x=testX, batch_size=BS)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=le.classes_))
# save the network to disk
print("[INFO] serializing network to '{}'...".format('/content/sample_data'))
model.save('/content/sample_data/myPBmodel')
#model.save('/content/sample_data/file1', save_format="h5")
# save the label encoder to disk
f = open('/content/sample_data/myleEncoder', "wb")
f.write(pickle.dumps(le))
f.close()
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, EPOCHS), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, EPOCHS), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, EPOCHS), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, EPOCHS), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig('/content/sample_data')

[INFO] evaluating network...
              precision    recall  f1-score   support

        fake       1.00      1.00      1.00        22
        real       1.00      1.00      1.00        38

    accuracy                           1.00        60
   macro avg       1.00      1.00      1.00        60
weighted avg       1.00      1.00      1.00        60

[INFO] serializing network to '/content/sample_data'...
INFO:tensorflow:Assets written to: /content/sample_data/myPBmodel/assets


In [109]:
# import the necessary packages
from imutils.video import VideoStream
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import argparse
import imutils
import pickle
import time
import cv2
import os

In [ ]:
# load our serialized face detector from disk
print("[INFO] loading face detector...")
#protoPath = os.path.sep.join([args["detector"], "deploy.prototxt"])
#modelPath = os.path.sep.join([args["detector"],"res10_300x300_ssd_iter_140000.caffemodel"])
net = cv2.dnn.readNetFromCaffe('/content/sample_data/deploy.prototxt.txt', '/content/sample_data/openCVfaceDetectorModel1.caffemodel')
# load the liveness detector model and label encoder from disk
print("[INFO] loading liveness detector...")
model = load_model('/content/sample_data/model')
le = pickle.loads(open('/content/sample_data/leEncoder', "rb").read())
# initialize the video stream and allow the camera sensor to warmup
print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()
time.sleep(2.0)

# loop over the frames from the video stream
while True:
	# grab the frame from the threaded video stream and resize it
	# to have a maximum width of 600 pixels
	frame = vs.read()
	frame = imutils.resize(frame, width=600)
	# grab the frame dimensions and convert it to a blob
	(h, w) = frame.shape[:2]
	blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0,
		(300, 300), (104.0, 177.0, 123.0))
	# pass the blob through the network and obtain the detections and
	# predictions
	net.setInput(blob)
	detections = net.forward()
  # loop over the detections
	for i in range(0, detections.shape[2]):
		# extract the confidence (i.e., probability) associated with the
		# prediction
		confidence = detections[0, 0, i, 2]
		# filter out weak detections
		if confidence > args["confidence"]:
			# compute the (x, y)-coordinates of the bounding box for
			# the face and extract the face ROI
			box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
			(startX, startY, endX, endY) = box.astype("int")
			# ensure the detected bounding box does fall outside the
			# dimensions of the frame
			startX = max(0, startX)
			startY = max(0, startY)
			endX = min(w, endX)
			endY = min(h, endY)
			# extract the face ROI and then preproces it in the exact
			# same manner as our training data
			face = frame[startY:endY, startX:endX]
			face = cv2.resize(face, (32, 32))
			face = face.astype("float") / 255.0
			face = img_to_array(face)
			face = np.expand_dims(face, axis=0)
			# pass the face ROI through the trained liveness detector
			# model to determine if the face is "real" or "fake"
			preds = model.predict(face)[0]
			j = np.argmax(preds)
			label = le.classes_[j]
			# draw the label and bounding box on the frame
			label = "{}: {:.4f}".format(label, preds[j])
			cv2.putText(frame, label, (startX, startY - 10),
				cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
			cv2.rectangle(frame, (startX, startY), (endX, endY),
				(0, 0, 255), 2)
   # show the output frame and wait for a key press
	cv2.imshow("Frame", frame)
	key = cv2.waitKey(1) & 0xFF
	# if the `q` key was pressed, break from the loop
	if key == ord("q"):
		break
# do a bit of cleanup
cv2.destroyAllWindows()
vs.stop()
      
      
 

In [118]:
import tensorflow as tf

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model('/content/sample_data/myPBmodel') # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)